In [9]:
!pip install pyserial

In [10]:
import serial

def read_data(ser):
    while True:
        if ser.in_waiting >= 7:
            data = ser.read(7)
            
            identifier = (data[0] >> 6) & 0x03
            analog_value = ((data[0] & 0x3F) << 4) | ((data[1] >> 4) & 0x0F)
            time_micros = (data[2] << 24) | (data[3] << 16) | (data[4] << 8) | data[5]
            
            print(f"Identifier: {identifier}, Analog Value: {analog_value}, Time: {time_micros}")

if __name__ == "__main__":
    ser = serial.Serial('/dev/cu.usbserial-110', 9600)  # Adjust the port as needed
    read_data(ser)


ModuleNotFoundError: No module named 'serial'